In [1]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/yy2895/Desktop/update_stresult19-15-19.csv',header=None)
d = df.values
ntotal=len(df)

# we do not renormalize it 
#d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(7)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,8):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 14
n_encoder_h_2 = 9



#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 14
n_decoder_h_2 = 19




# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=7
training_epochs = 5000

In [2]:
#we store the variables here
log_files_path = 'C:/Users/yy2895/Desktop/st15-12-15/tmp/model.ckpt'
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [15, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, 15], [15], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '12'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 15]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive100=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/yy2895/Desktop/st15-12-15/tmp/model.ckpt')
                        togive4=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,15)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive4.append(output_any_image)
                            
                        togive100=togive4
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 1.443781137
Validation Loss: 1.946187
Epoch: 0002 cost = 1.423898407
Validation Loss: 1.6862603
Epoch: 0003 cost = 1.404661826
Validation Loss: 1.5295599
Epoch: 0004 cost = 1.386461854
Validation Loss: 1.4572841
Epoch: 0005 cost = 1.368692483
Validation Loss: 1.460228
Epoch: 0006 cost = 1.349577427
Validation Loss: 1.3358514
Epoch: 0007 cost = 1.325785245
Validation Loss: 1.4209105
Epoch: 0008 cost = 1.294718266
Validation Loss: 1.3363613
Epoch: 0009 cost = 1.265867404
Validation Loss: 1.370359
Epoch: 0010 cost = 1.244474207
Validation Loss: 1.4007187
Epoch: 0011 cost = 1.225911209
Validation Loss: 1.4136133
Epoch: 0012 cost = 1.208220788
Validation Loss: 1.4184484
Epoch: 0013 cost = 1.191461495
Validation Loss: 1.397293
Epoch: 0014 cost = 1.176175475
Validation Loss: 1.3464154
Epoch: 0015 cost = 1.162802202
Validation Loss: 1.2957115
Epoch: 0016 cost = 1.151400396
Validation Loss: 1.3906928
Epoch: 0017 cost = 1.141514761
Validation Loss: 1.2742362
Epoch: 00

Epoch: 0143 cost = 0.464848761
Validation Loss: 0.51093715
Epoch: 0144 cost = 0.461453706
Validation Loss: 0.44614476
Epoch: 0145 cost = 0.458080756
Validation Loss: 0.43623766
Epoch: 0146 cost = 0.454729732
Validation Loss: 0.38421693
Epoch: 0147 cost = 0.451400578
Validation Loss: 0.4702245
Epoch: 0148 cost = 0.448093606
Validation Loss: 0.32868695
Epoch: 0149 cost = 0.444808653
Validation Loss: 0.41298303
Epoch: 0150 cost = 0.441546547
Validation Loss: 0.37387058
Epoch: 0151 cost = 0.438307907
Validation Loss: 0.33967718
Epoch: 0152 cost = 0.435093654
Validation Loss: 0.34887877
Epoch: 0153 cost = 0.431905202
Validation Loss: 0.3597652
Epoch: 0154 cost = 0.428742984
Validation Loss: 0.3349866
Epoch: 0155 cost = 0.425608380
Validation Loss: 0.36736074
Epoch: 0156 cost = 0.422502420
Validation Loss: 0.36983028
Epoch: 0157 cost = 0.419425918
Validation Loss: 0.32891366
Epoch: 0158 cost = 0.416379141
Validation Loss: 0.3387396
Epoch: 0159 cost = 0.413362818
Validation Loss: 0.34980032
E

Validation Loss: 0.3701964
Epoch: 0297 cost = 0.181817936
Validation Loss: 0.42631727
Epoch: 0298 cost = 0.180786139
Validation Loss: 0.4135653
Epoch: 0299 cost = 0.179759266
Validation Loss: 0.31065807
Epoch: 0300 cost = 0.178737713
Validation Loss: 0.26357776
Epoch: 0301 cost = 0.177721792
Validation Loss: 0.29003426
Epoch: 0302 cost = 0.176711532
Validation Loss: 0.29055715
Epoch: 0303 cost = 0.175707532
Validation Loss: 0.18010844
Epoch: 0304 cost = 0.174709908
Validation Loss: 0.28922844
Epoch: 0305 cost = 0.173718793
Validation Loss: 0.2032914
Epoch: 0306 cost = 0.172734916
Validation Loss: 0.26143888
Epoch: 0307 cost = 0.171757524
Validation Loss: 0.23692271
Epoch: 0308 cost = 0.170787394
Validation Loss: 0.23941104
Epoch: 0309 cost = 0.169824558
Validation Loss: 0.33599737
Epoch: 0310 cost = 0.168869244
Validation Loss: 0.28907207
Epoch: 0311 cost = 0.167921432
Validation Loss: 0.29743528
Epoch: 0312 cost = 0.166981188
Validation Loss: 0.27973205
Epoch: 0313 cost = 0.166048874


Validation Loss: 0.16474444
Epoch: 0449 cost = 0.088869389
Validation Loss: 0.11458719
Epoch: 0450 cost = 0.088464921
Validation Loss: 0.11288174
Epoch: 0451 cost = 0.088075907
Validation Loss: 0.115612976
Epoch: 0452 cost = 0.087701637
Validation Loss: 0.117705904
Epoch: 0453 cost = 0.087339665
Validation Loss: 0.09359282
Epoch: 0454 cost = 0.086986216
Validation Loss: 0.110585526
Epoch: 0455 cost = 0.086638001
Validation Loss: 0.11428457
Epoch: 0456 cost = 0.086292568
Validation Loss: 0.12347729
Epoch: 0457 cost = 0.085948820
Validation Loss: 0.1126035
Epoch: 0458 cost = 0.085606298
Validation Loss: 0.07634462
Epoch: 0459 cost = 0.085264120
Validation Loss: 0.13648961
Epoch: 0460 cost = 0.084920957
Validation Loss: 0.09145202
Epoch: 0461 cost = 0.084575409
Validation Loss: 0.081381105
Epoch: 0462 cost = 0.084225550
Validation Loss: 0.07485068
Epoch: 0463 cost = 0.083871612
Validation Loss: 0.1330285
Epoch: 0464 cost = 0.083515796
Validation Loss: 0.0807082
Epoch: 0465 cost = 0.083161

Validation Loss: 0.09269849
Epoch: 0590 cost = 0.048081930
Validation Loss: 0.06846194
Epoch: 0591 cost = 0.047849770
Validation Loss: 0.056269594
Epoch: 0592 cost = 0.047615914
Validation Loss: 0.12929656
Epoch: 0593 cost = 0.047380514
Validation Loss: 0.08418613
Epoch: 0594 cost = 0.047143668
Validation Loss: 0.07349908
Epoch: 0595 cost = 0.046905288
Validation Loss: 0.071986936
Epoch: 0596 cost = 0.046667220
Validation Loss: 0.062352322
Epoch: 0597 cost = 0.046432502
Validation Loss: 0.065334275
Epoch: 0598 cost = 0.046203515
Validation Loss: 0.06170659
Epoch: 0599 cost = 0.045980177
Validation Loss: 0.070148595
Epoch: 0600 cost = 0.045760887
Validation Loss: 0.08659053
Epoch: 0601 cost = 0.045543223
Validation Loss: 0.057232823
Epoch: 0602 cost = 0.045325593
Validation Loss: 0.053762145
Epoch: 0603 cost = 0.045108367
Validation Loss: 0.10616895
Epoch: 0604 cost = 0.044894002
Validation Loss: 0.060881414
Epoch: 0605 cost = 0.044690482
Validation Loss: 0.05549785
Epoch: 0606 cost = 0

...

Validation Loss: 0.035414997
Epoch: 4404 cost = 0.017230354
Validation Loss: 0.024015918
Epoch: 4405 cost = 0.017229545
Validation Loss: 0.022715868
Epoch: 4406 cost = 0.017228887
Validation Loss: 0.028696582
Epoch: 4407 cost = 0.017228519
Validation Loss: 0.047990035
Epoch: 4408 cost = 0.017227695
Validation Loss: 0.04507647
Epoch: 4409 cost = 0.017227427
Validation Loss: 0.04127007
Epoch: 4410 cost = 0.017226857
Validation Loss: 0.037938055
Epoch: 4411 cost = 0.017226110
Validation Loss: 0.06200159
Epoch: 4412 cost = 0.017225984
Validation Loss: 0.08700636
Epoch: 4413 cost = 0.017225267
Validation Loss: 0.08848346
Epoch: 4414 cost = 0.017224813
Validation Loss: 0.08236118
Epoch: 4415 cost = 0.017224205
Validation Loss: 0.04947316
Epoch: 4416 cost = 0.017224043
Validation Loss: 0.028670523
Epoch: 4417 cost = 0.017223049
Validation Loss: 0.03433495
Epoch: 4418 cost = 0.017222860
Validation Loss: 0.025429605
Epoch: 4419 cost = 0.017222590
Validation Loss: 0.040067352
Epoch: 4420 cost = 

Epoch: 4552 cost = 0.017155877
Validation Loss: 0.034749657
Epoch: 4553 cost = 0.017155672
Validation Loss: 0.026659524
Epoch: 4554 cost = 0.017155170
Validation Loss: 0.016860964
Epoch: 4555 cost = 0.017154849
Validation Loss: 0.06614633
Epoch: 4556 cost = 0.017153928
Validation Loss: 0.058523633
Epoch: 4557 cost = 0.017153584
Validation Loss: 0.045580257
Epoch: 4558 cost = 0.017153068
Validation Loss: 0.045733184
Epoch: 4559 cost = 0.017152381
Validation Loss: 0.039136183
Epoch: 4560 cost = 0.017151945
Validation Loss: 0.04104745
Epoch: 4561 cost = 0.017151353
Validation Loss: 0.043806348
Epoch: 4562 cost = 0.017151405
Validation Loss: 0.059882022
Epoch: 4563 cost = 0.017151015
Validation Loss: 0.074386194
Epoch: 4564 cost = 0.017150187
Validation Loss: 0.048342686
Epoch: 4565 cost = 0.017149580
Validation Loss: 0.03892628
Epoch: 4566 cost = 0.017148871
Validation Loss: 0.04160676
Epoch: 4567 cost = 0.017148928
Validation Loss: 0.077997744
Epoch: 4568 cost = 0.017148028
Validation Lo

Validation Loss: 0.054820843
Epoch: 4704 cost = 0.017084303
Validation Loss: 0.033345733
Epoch: 4705 cost = 0.017084101
Validation Loss: 0.034372255
Epoch: 4706 cost = 0.017083292
Validation Loss: 0.038018286
Epoch: 4707 cost = 0.017083091
Validation Loss: 0.064770654
Epoch: 4708 cost = 0.017082450
Validation Loss: 0.09540022
Epoch: 4709 cost = 0.017081953
Validation Loss: 0.10869691
Epoch: 4710 cost = 0.017081502
Validation Loss: 0.079177216
Epoch: 4711 cost = 0.017080489
Validation Loss: 0.032293953
Epoch: 4712 cost = 0.017080033
Validation Loss: 0.027645856
Epoch: 4713 cost = 0.017079822
Validation Loss: 0.052716102
Epoch: 4714 cost = 0.017079188
Validation Loss: 0.07680865
Epoch: 4715 cost = 0.017079094
Validation Loss: 0.04948392
Epoch: 4716 cost = 0.017078554
Validation Loss: 0.044803865
Epoch: 4717 cost = 0.017078106
Validation Loss: 0.06430885
Epoch: 4718 cost = 0.017077844
Validation Loss: 0.042828567
Epoch: 4719 cost = 0.017076672
Validation Loss: 0.03616513
Epoch: 4720 cost 

Validation Loss: 0.0889287
Epoch: 4851 cost = 0.017017830
Validation Loss: 0.10221798
Epoch: 4852 cost = 0.017017400
Validation Loss: 0.06581681
Epoch: 4853 cost = 0.017017435
Validation Loss: 0.046667945
Epoch: 4854 cost = 0.017016777
Validation Loss: 0.04237946
Epoch: 4855 cost = 0.017016823
Validation Loss: 0.032572456
Epoch: 4856 cost = 0.017016489
Validation Loss: 0.02775594
Epoch: 4857 cost = 0.017015836
Validation Loss: 0.030605055
Epoch: 4858 cost = 0.017015909
Validation Loss: 0.03399375
Epoch: 4859 cost = 0.017015529
Validation Loss: 0.038357336
Epoch: 4860 cost = 0.017014655
Validation Loss: 0.051539022
Epoch: 4861 cost = 0.017014153
Validation Loss: 0.04771665
Epoch: 4862 cost = 0.017014182
Validation Loss: 0.036098294
Epoch: 4863 cost = 0.017013265
Validation Loss: 0.057340004
Epoch: 4864 cost = 0.017013581
Validation Loss: 0.067552425
Epoch: 4865 cost = 0.017012273
Validation Loss: 0.054881167
Epoch: 4866 cost = 0.017011975
Validation Loss: 0.05808888
Epoch: 4867 cost = 0

Epoch: 4995 cost = 0.016948580
Validation Loss: 0.03447316
Epoch: 4996 cost = 0.016942941
Validation Loss: 0.03203214
Epoch: 4997 cost = 0.016937794
Validation Loss: 0.04298062
Epoch: 4998 cost = 0.016934123
Validation Loss: 0.0467359
Epoch: 4999 cost = 0.016930196
Validation Loss: 0.053728245
Epoch: 5000 cost = 0.016927041
Validation Loss: 0.04203015
Optimization Finished!
Test Loss: 0.039048117


In [3]:
currentmin

0.016860964

In [6]:
import csv
with open('C:/Users/yy2895/Desktop/update_stresult15-12-15.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive100)):
        wr.writerow(list(togive100[i][0]))

In [5]:
togive100

[array([[0.23245826, 0.17940712, 0.14006864, 0.5881337 , 0.19452874,
         0.8957805 , 0.17331076, 0.89777666, 0.8822948 , 0.8448451 ,
         0.2797415 , 0.9086028 ]], dtype=float32),
 array([[0.23581468, 0.1842496 , 0.15721422, 0.58996737, 0.19807823,
         0.88539183, 0.17599826, 0.8926599 , 0.86924213, 0.839592  ,
         0.26024953, 0.9032815 ]], dtype=float32),
 array([[0.28703323, 0.18859379, 0.1761419 , 0.63856745, 0.20452034,
         0.86446506, 0.25961348, 0.85537606, 0.84788156, 0.7850526 ,
         0.26088893, 0.8684034 ]], dtype=float32),
 array([[0.36432374, 0.20287919, 0.19722061, 0.5997979 , 0.22918402,
         0.8440207 , 0.39524326, 0.8101399 , 0.8236773 , 0.7233658 ,
         0.29931876, 0.8277678 ]], dtype=float32),
 array([[0.4522251 , 0.20713754, 0.20780112, 0.6194414 , 0.24215764,
         0.82837296, 0.5666883 , 0.7346474 , 0.8076063 , 0.63859695,
         0.3375831 , 0.75922585]], dtype=float32),
 array([[0.35054812, 0.1977856 , 0.21370697, 0.63403136